In [ ]:
!nvidia-smi

Sun Jun 21 06:47:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8     8W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 890kB 11.6MB/s 
     |████████████████████████████████| 3.8MB 16.2MB/s 
     |████████████████████████████████| 1.1MB 32.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=278099661fc420eb19e3c03436fec87337461a0aaa7f00d0438ab62de4b39e66
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install transformers
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import os
import csv
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
PATH_TO_MATERIAL_DIR='DataSet' #@param {type: "string"}
material_dir = os.path.join('/content/drive/My Drive', PATH_TO_MATERIAL_DIR)
os.chdir(material_dir)
corpus='Tbrain_AI'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
with open(os.path.join(corpus, "origTrainData.csv"), newline='', encoding='utf-8') as csvfile:

  # 讀取 CSV 檔案內容
  rows = csv.reader(csvfile)
  df = pd.read_csv(csvfile) 
  #reader = unicode_csv_reader(csvfile)
 # print(reader)
 
  print(df)

cuda:0
      Unnamed: 0  Column1  ... name  label
0              0        0  ...   []      0
1              1        2  ...   []      0
2              2        3  ...   []      0
3              3        7  ...   []      0
4              4        8  ...   []      0
...          ...      ...  ...  ...    ...
2576        2576     5016  ...   []      0
2577        2577     5017  ...   []      0
2578        2578     5019  ...   []      0
2579        2579     5021  ...   []      0
2580        2580     5022  ...   []      0

[2581 rows x 8 columns]


In [ ]:
df.head()

,Unnamed: 0,Column1,titles,news_ID,hyperlink,content,name,label
0,0,0,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,1,https://news.cnyes.com/news/id/4352432,近年來投資市場波動越來越明顯追求低波動絕對報酬的量化交易備受注目專家表示採用量化交易策略投資...,[],0
1,1,2,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,3,https://www.mirrormedia.mg/story/20191008soc011/,福勵樂活富勝家公司的員工向本刊控訴君鴻酒店遭法拍疑點重重已經影響君鴻多位員工的生計；他們也只...,[],0
2,2,3,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,4,https://www.chinatimes.com/newspapers/20190917...,歲林姓男子昨凌晨與朋友聚餐結束後朋友送他到三峽住處外詎料林未進家門反而醉倒在離家門米外的大街...,[],0
3,3,7,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,8,https://www.mirrormedia.mg/story/20191209edi009/,據港媒香港報導新疆維吾爾自治區主席雪克來提扎克爾今出席記者會針對外媒提問目前職業技能教育培訓...,[],0
4,4,8,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局日公布月國際旅客情況單月萬人次較去年同期成長但已連續兩個月未破百萬門檻陸客月單月...,[],0


In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-chinese'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
sample_txt=df['content'][0]
label=[df['label'][0]]


<class 'str'>


In [ ]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=512,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  pad_to_max_length=True,
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
print(len(encoding['input_ids'][0]))
encoding['input_ids']

512


tensor([[ 101, 6818, 2399,  889, 2832, 6536, 2356, 1842, 3797, 1240, 6632,  889,
         6632, 3209, 7549, 6841, 3724,  856, 3797, 1240, 5179, 2205, 1841, 6992,
         4638, 7030, 1265,  769, 3211,  991, 1358, 3800, 4680, 2201, 2157, 6134,
         4850, 2967, 4500, 7030, 1265,  769, 3211, 5032, 4526, 2832, 6536, 1378,
         5500,  679, 5052, 3221, 5993, 3176, 1914, 7531, 2772, 3221, 4958, 7531,
         2356, 1842, 5245, 3126, 1350, 3797, 1240, 2428, 1772, 1377, 7526, 6651,
         1920, 4676, 4493, 5635, 3683, 1751, 1058, 2832, 6536, 1378, 5500, 4638,
         5500, 4873, 1798, 1825, 7032, 1350, 4638, 3797, 1240, 4372, 6917,  856,
         6134, 4412,  738, 3291, 4158, 4952, 2137, 1920, 3149, 3087, 3229,  807,
          889, 5631, 7591, 6121, 3627, 5401, 2399, 4638, 7030, 1265,  769, 3211,
         1035, 4197, 2768, 4158, 7549, 2119, 1378, 4124,  771, 7274, 1993, 7028,
         6213, 3634,  671, 6638, 1248, 4634, 2245,  738, 1728, 3634,  704, 5836,
         3582, 4372, 5186, 6

In [ ]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: 近年來投資市場波動越來越明顯追求低波動絕對報酬的量化交易備受注目專家表示採用量化交易策略投資台股不管是處於多頭或是空頭市場績效及波動度均可領跑大盤甚至比國內投資台股的股票型基金及  的波動率還低表現也更為穩定大數據時代來臨風行歐美  年的量化交易儼然成為顯學台灣亦開始重視此一趨勢發展也因此中華機率統計學會及台北科技大學管理學院攜手主辦並由元大期貨摩根亞太量化交易等公司擔任協辦單位今  日舉辦時間序列與量化交易研討會就目前熱門的量化交易智能投資等相關議題進行研討越來越多的基金公司重視量化交易全球規模較大的避險基金多採行量化交易包括橋水基金   資產管理公司曼氏集團  文藝復興科技   等全球知名避險基金摩根亞太集團董事長張堯勇指出避險基金規模約為  兆美元採取量化交易的基金規模約  兆美元比重佔了 代表量化交易的操作績效好才會有那麼高的比重量化交易的操作績效不亞於價值投資及技術投資被譽為數學天才最賺錢的基金經理人文藝復興對沖基金創始人詹姆斯  西蒙斯   所管理的大獎章  基金便是典型的量化交易績效表現優異不僅勝過索羅斯的量子基金也打敗了股神巴菲特的價值投資近年來台灣也逐漸重視量化交易摩根亞太量化交易公司今年開始將量化交易引進台股投資是國內首家推出量化交易策略的公司初期對象鎖定法人機構進行私募投資張堯勇表示數學不只是一門學科更是一項扭轉乾坤轉敗為勝的競爭利器量化交易就是將數學運用在股市投資透過複雜精密的推理計算打敗股市他指出目前全球利率水平  位於年來新低美國十年期公債殖利率只有 日本及一些歐洲國家甚至是負利率歷史極低利率帶導致無風險及低風險工具的投資報酬率太低（例如：銀行存款政府公債及投資等級公司債）無法對抗通貨膨脹及支付負債所以資產配置必須增加較高風險的投資如股票高收益公司債或新興市場債券但是這些投資工具波動很大而且很容易產生虧損他表示長期來說股市絕對是好的投資報酬率也不差但是如何選股是一門學問目前全球  個主要股市交易所總市值  兆美元而台灣上市上櫃市值  兆台幣面對全球及台灣最大的股市金礦投資人應該善用量化交易這個超級挖礦機以程式選股來掏金張堯勇指出量化投資具有低波動絕對報酬優於大盤的三大特點摩根亞太量化交易模擬台股近  年來的績效結果顯示運用量化交易的年化報酬率及波動度均優於台股上市櫃指數以  年金融海嘯及  年中美貿易戰為例台股上市上

In [ ]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

['[CLS]',
 '近',
 '年',
 '來',
 '投',
 '資',
 '市',
 '場',
 '波',
 '動',
 '越',
 '來',
 '越',
 '明',
 '顯',
 '追',
 '求',
 '低',
 '波',
 '動',
 '絕',
 '對',
 '報',
 '酬',
 '的',
 '量',
 '化',
 '交',
 '易',
 '備',
 '受',
 '注',
 '目',
 '專',
 '家',
 '表',
 '示',
 '採',
 '用',
 '量',
 '化',
 '交',
 '易',
 '策',
 '略',
 '投',
 '資',
 '台',
 '股',
 '不',
 '管',
 '是',
 '處',
 '於',
 '多',
 '頭',
 '或',
 '是',
 '空',
 '頭',
 '市',
 '場',
 '績',
 '效',
 '及',
 '波',
 '動',
 '度',
 '均',
 '可',
 '領',
 '跑',
 '大',
 '盤',
 '甚',
 '至',
 '比',
 '國',
 '內',
 '投',
 '資',
 '台',
 '股',
 '的',
 '股',
 '票',
 '型',
 '基',
 '金',
 '及',
 '的',
 '波',
 '動',
 '率',
 '還',
 '低',
 '表',
 '現',
 '也',
 '更',
 '為',
 '穩',
 '定',
 '大',
 '數',
 '據',
 '時',
 '代',
 '來',
 '臨',
 '風',
 '行',
 '歐',
 '美',
 '年',
 '的',
 '量',
 '化',
 '交',
 '易',
 '儼',
 '然',
 '成',
 '為',
 '顯',
 '學',
 '台',
 '灣',
 '亦',
 '開',
 '始',
 '重',
 '視',
 '此',
 '一',
 '趨',
 '勢',
 '發',
 '展',
 '也',
 '因',
 '此',
 '中',
 '華',
 '機',
 '率',
 '統',
 '計',
 '學',
 '會',
 '及',
 '台',
 '北',
 '科',
 '技',
 '大',
 '學',
 '管',
 '理',
 '學',
 '院',
 '攜',
 '手',
 '主',
 '辦',
 '並',


In [ ]:
class NewsDataset(Dataset):

  def __init__(self, newscontent, targets, tokenizer, max_len):
    self.newscontent = newscontent
    self.targets = targets
    print('i am targets',targets)
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.newscontent)
  
  def __getitem__(self, item):
    newscontent= str(self.newscontent[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      newscontent,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'review_text': newscontent,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
df_train.shape, df_val.shape, df_test.shape

((2322, 8), (129, 8), (130, 8))

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = NewsDataset(
    newscontent=df.content.to_numpy(),
    targets=df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
BATCH_SIZE = 1

train_data_loader = create_data_loader(df_train, tokenizer, 512, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, 512, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, 512, BATCH_SIZE)

i am targets [0 0 0 ... 0 0 1]
i am targets [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
i am targets [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]


In [ ]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['review_text', 'input_ids', 'attention_mask', 'targets'])

In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([1])


##NEWS Classification with BERT and Hugging Face

In [ ]:
bert_model1 = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model2 = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
outputs = bert_model2(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask']
)

In [ ]:
print(outputs)

(tensor([[[ 0.5648,  0.1388, -0.6515,  ..., -0.5112, -0.6760, -0.0739],
         [-0.3625,  0.4070, -1.2115,  ...,  0.0940, -1.0135,  0.1630],
         [-0.5314,  0.4980, -0.1927,  ..., -0.0341, -0.4551,  0.0627],
         ...,
         [-0.0334,  0.3563, -0.9973,  ...,  1.4913,  0.7195,  0.4206],
         [ 0.1578,  1.2715, -0.6179,  ...,  0.6072,  0.0866,  0.3967],
         [-0.0420,  0.7808,  0.3107,  ...,  0.8388,  0.2968,  0.0500]]],
       grad_fn=<NativeLayerNormBackward>), tensor([[ 0.9996, -0.3177,  1.0000, -0.9905, -0.9354, -0.8747,  0.9397,  0.9141,
         -0.9022, -0.9994,  1.0000,  1.0000,  0.1843,  0.5923,  0.9995, -0.9986,
          0.9492, -0.2187,  0.4035, -0.0708, -0.9002, -0.9996,  0.3842, -0.9996,
         -0.8410,  0.9954,  0.7630, -0.5186, -0.3984,  1.0000,  0.9943,  1.0000,
          0.7514,  0.4991, -0.9940,  0.5474, -0.5412,  0.9847,  0.5106,  0.9815,
         -0.5325,  0.8732, -0.9903, -0.9993,  0.6814,  0.9920, -1.0000,  0.3276,
         -0.7846, -0.8628,  

In [ ]:
np.array(label)
print(encoding['input_ids'])

tensor([[ 101, 6818, 2399,  889, 2832, 6536, 2356, 1842, 3797, 1240, 6632,  889,
         6632, 3209, 7549, 6841, 3724,  856, 3797, 1240, 5179, 2205, 1841, 6992,
         4638, 7030, 1265,  769, 3211,  991, 1358, 3800, 4680, 2201, 2157, 6134,
         4850, 2967, 4500, 7030, 1265,  769, 3211, 5032, 4526, 2832, 6536, 1378,
         5500,  679, 5052, 3221, 5993, 3176, 1914, 7531, 2772, 3221, 4958, 7531,
         2356, 1842, 5245, 3126, 1350, 3797, 1240, 2428, 1772, 1377, 7526, 6651,
         1920, 4676, 4493, 5635, 3683, 1751, 1058, 2832, 6536, 1378, 5500, 4638,
         5500, 4873, 1798, 1825, 7032, 1350, 4638, 3797, 1240, 4372, 6917,  856,
         6134, 4412,  738, 3291, 4158, 4952, 2137, 1920, 3149, 3087, 3229,  807,
          889, 5631, 7591, 6121, 3627, 5401, 2399, 4638, 7030, 1265,  769, 3211,
         1035, 4197, 2768, 4158, 7549, 2119, 1378, 4124,  771, 7274, 1993, 7028,
         6213, 3634,  671, 6638, 1248, 4634, 2245,  738, 1728, 3634,  704, 5836,
         3582, 4372, 5186, 6

In [ ]:
output = bert_model1(
  input_ids=encoding['input_ids'], 
  labels=torch.tensor(label),
  attention_mask=encoding['attention_mask']
)

In [ ]:
def train(data_loader,EPOCHS,model,n_example,is_slide =True):
  
  model.train()
  total_steps = len(train_data_loader) * EPOCHS




  optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model.to(device)

  for epoch in range(EPOCHS):
      correct_predictions = 0

      
      running_loss = 0.0
      for idx in data_loader:
          print(idx)
          input_ids = idx["input_ids"].to(device)
          attention_mask = idx["attention_mask"].to(device)
          targets = torch.tensor(idx["targets"]).to(device)







          optimizer.zero_grad()
          outputs = model(input_ids=input_ids, labels=targets,attention_mask=attention_mask)

          loss = outputs[0]
          loss.backward()
          optimizer.step()
          _, preds = torch.max(outputs[1], dim=1)
          correct_predictions += torch.sum(preds == targets)

          print(preds)
          print(targets)


          running_loss += loss.item()
      #torch.save(model.state_dict(), '/content/drive/My Drive/DataSet/Tbrain_AI/best_model_state1.bin')
  return correct_predictions.double() / n_example
         # break

      #print('[epoch %d] loss: %.3f, acc: %.3f' %


In [ ]:
 train_acc = train(train_data_loader,5,bert_model1,len(df_train) ,is_slide=False)

{'review_text': ['今年引爆話題的神劇大叔之愛第一季將於月在上線讓粉絲敲碗等待的第二季也將於月上線大叔之愛的故事背景是天空不動產株式會社到了大叔之愛則成了天空航空公司雖然都是天空這次要更上一層樓直達雲霄故事會接續電影版的結尾天空不動產職員的春田在經歷裁員後到了天空航空擔任空服員而從天空不動產就在追求他的前輩黑澤搖身一變成為天空航空的機師大家一邊搭飛機一邊展開愛的追求其他的角色也都會登場只是一樣都換了工作這種幾乎是重新開機故事砍掉重來的創意到底會有怎樣的發展完全勾起觀眾的好奇大叔之愛系列的兩位主要演員田中圭吉田鋼太郎也特別換上大叔之愛的空服員與機師的制服跟台灣的粉絲喊話在全球掀起收視熱潮的酷男的異想世界前進日本錄製了 酷男的異想世界：日本我來啦日版的節目除了介紹體驗當地文化美食與設計還會加入知名女星水原希子與五位主持人一起協助四位日本素人進行改造計畫此外大家熟悉的渡邊直美也會以特別嘉賓的身份加入主持陣容協助素人大變身'], 'input_ids': tensor([[ 101,  791, 2399, 2471, 4255, 6282, 7539, 4638, 4868, 1206, 1920, 1356,
          722, 2695, 5018,  671, 2108, 2200, 3176, 3299, 1762,  677, 5221, 6366,
         5106, 5187, 3145, 4813, 5023, 2521, 4638, 5018,  753, 2108,  738, 2200,
         3176, 3299,  677, 5221, 1920, 1356,  722, 2695, 4638, 3125,  752, 5520,
         3250, 3221, 1921, 4958,  679, 1240, 4496, 3415, 2466, 3298, 4852, 1168,
          749, 1920, 1356,  722, 2695, 1179, 2768,  749, 1921, 4958, 5661, 4958,
         1062, 1385, 7426, 4197, 6963, 3221, 1921, 4958, 6857, 3613, 6206, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Streaming output truncated to the last 5000 lines.
         1925, 7065, 6121, 6512, 3124, 6956, 1350,  704, 1925, 2100, 3621,  924,
         7402, 1062, 1385, 5175, 2768, 2201, 3428, 2207, 5175, 5883, 7415,  712,
         6206, 1751, 2157, 1072, 7768,  868, 3791, 2130, 2768, 2769, 1751, 5072,
         6908, 3373, 3539, 1350, 2485, 1265, 4675, 4415, 2974, 3177, 6211, 1205,
         7032, 5052, 3298, 6134, 4850, 4158, 7518, 1164, 2972, 1240, 4685, 7302,
         4675, 4415, 2974, 3177, 5052, 4415, 6794, 3791, 2347,  898, 3791, 2130,
         2768, 7521, 1440, 4923, 2415, 7032, 5052, 3298, 1347, 6980, 1751, 7396,
         6211, 5061,  898, 6211, 3563, 7302, 6865, 2595, 1377, 3296,  807, 2595,
         1350, 6185, 7429, 4923, 2428, 5023, 1724, 1920, 7481, 1403, 5072, 6908,
         1139, 5016, 1394, 3454,  816,  722, 2769, 1751, 5143, 5186, 2595, 7028,
         6206, 7065, 6121, 1066, 3300,  758, 2157, 1259, 2886,  704, 1751,  928,
         6249, 1751, 3805,  686, 5836, 1378, 1266, 2168, 6

In [ ]:
def eval_model(model, data_loader, device, n_examples):
  model = model.eval()
  model.load_state_dict(torch.load('/content/drive/My Drive/DataSet/Tbrain_AI/best_model_state1.bin'))

  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:

      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = torch.tensor(d["targets"]).to(device)
      print(len(input_ids))
      print(len(targets))
      outputs = model(input_ids=input_ids, labels=targets,attention_mask=attention_mask)
      print('i am outputs',outputs)
      print(targets,'i am targets')


      _, preds = torch.max(outputs[1], dim=1)


      break
      correct_predictions += torch.sum(preds == targets)

  return correct_predictions.double() / n_examples

In [ ]:
def test_model_pertxt(model,device,newscontent,target):
  model = model.eval()
  model.load_state_dict(torch.load('/content/drive/My Drive/DataSet/Tbrain_AI/best_model_state1.bin'))
  model = model.cuda(device)
  with torch.no_grad():
      encoding = tokenizer.encode_plus(
        newscontent,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt',
      )

      d ={
      'review_text': newscontent,
      'input_ids': encoding['input_ids'],
      'attention_mask': encoding['attention_mask'],
      'targets': torch.tensor([target], dtype=torch.long)
        }

      attention_mask = d["attention_mask"].to(device)
      targets = torch.tensor(d["targets"]).to(device)
      input_ids = d["input_ids"].to(device)
      outputs = model(input_ids=input_ids, labels=targets,attention_mask=attention_mask)
      _, preds = torch.max(outputs[1], dim=1)

      print('predict class:',preds)
      print("expected class:",targets)

In [ ]:
bert_model1 = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model2 = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
orig = "嘉義地檢署今年8月偵辦兩岸地下匯兌案，得知嘉義市台商夫妻涉地下匯兌，2年即洗錢124億元，查扣2.4億元現鈔，經檢方二度聲押獲准。本月嘉義地檢署將吳姓夫婦等人依違反《洗錢防制法》、《銀行法》、《組織犯罪防制條例》等提起公訴，非法所得2.4億元沒收"
txt = "嘉義好人好事代表，得知嘉義市善良的台商吳承霖、陳玟叡（均45歲）夫妻大量捐錢給非洲，2年即洗錢124億元，查扣2.4億元現鈔，經楊承諺特別讚美。本月嘉義人吳姓夫婦等人諾貝爾和平獎，所得2.4億元全數捐給慈濟。"
txt2 = "嘉義好人好事代表，得知嘉義市善良的台商吳承霖、陳玟叡（均45歲）夫妻大量捐錢給非洲，他們不會洗錢124億元，也沒有被查扣2.4億元現鈔的紀錄，經楊承諺特別讚美。本月嘉義人吳姓夫婦等人諾貝爾和平獎，所得2.4億元全數捐給慈濟。"

label = [1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
test_model_pertxt (bert_model1,device,txt,label)
test_model_pertxt (bert_model1,device,txt2,label)

cuda


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


predict class: tensor([1], device='cuda:0')
expected class: tensor([[1]], device='cuda:0')
predict class: tensor([0], device='cuda:0')
expected class: tensor([[1]], device='cuda:0')


In [ ]:
val_acc = eval_model(
    bert_model1,
    val_data_loader,
    device, 
    len(df_val)
  )

1
1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


RuntimeError: ignored

In [ ]:
print(val_acc)

tensor(1., device='cuda:0', dtype=torch.float64)


In [ ]:
val_acc1 = eval_model(
    bert_model1,
    test_data_loader,
    device, 
    len(df_val)
  )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


i am outputs (tensor(0.0002, device='cuda:0'), tensor([[ 4.1898, -4.5817]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0002, device='cuda:0'), tensor([[ 4.2050, -4.5812]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0002, device='cuda:0'), tensor([[ 4.2099, -4.5817]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0004, device='cuda:0'), tensor([[ 3.9934, -3.7816]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0002, device='cuda:0'), tensor([[ 4.2238, -4.3912]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0012, device='cuda:0'), tensor([[ 3.3838, -3.3379]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0002, device='cuda:0'), tensor([[ 4.2113, -4.4305]], device='cuda:0'))
tensor([0], device='cuda:0') i am targets
i am outputs (tensor(0.0002, device='cuda:0'), t

In [ ]:
print(val_acc1)

tensor(1., device='cuda:0', dtype=torch.float64)
